In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from kmodes.kmodes import KModes
import functions

sns.set_style('whitegrid')

In [2]:
data_full = pd.read_csv("./data/generated/Analizsiz_v1.3.csv")
data = data_full.sample(1000)

In [3]:
def apply_worker_type(row):
    try:
        return row["newpositiongroupcode"][0]
    except:
        return None
data['worker_type'] = data.apply(apply_worker_type,axis=1)

def apply_level(row):
    try:
        level = row["newpositiongroupcode"][1]
        if level == "O":
            return 0
        return int(level)
    except:
        return -1
    
data['expertise'] = data.apply(apply_level,axis=1)

In [4]:
data = data[["classification", "positiondescription", "mainsectorname","companyId","worker_type","expertise"]]
data.rename(
    columns = {
        'classification':'position_sector', 
        'positiondescription':'position_name',
        "mainsectorname": "company_sector",
        "companyId": "company_id"
    }, 
    inplace = True
)
data = data.dropna()

data = data.drop(["company_id"],axis=1)

data

,position_sector,position_name,company_sector,worker_type,expertise
347742,Müşteri Hizmetleri,Müşteri Destek Sorumlusu,Hizmet,B,2
339829,Kamu,Sosyal Güvenlik Denetmeni,Reklam ve Tanıtım,B,2
219663,İnşaat,Döşeme Ustası,Mobilya & Aksesuar,M,3
68839,Basım Yayın / Matbaa,Baskı Operatörü,Ofis / Büro Malzemeleri,M,2
363054,Denetim,Denetçi Yardımcısı,Hizmet,B,2
...,...,...,...,...,...
423888,Otomotiv,Filo Yöneticisi / Yönetmeni,Ticaret,B,3
132734,Üretim,Üretim Teknisyeni,Üretim / Endüstriyel Ürünler,M,2
190503,Taşımacılık / Lojistik,Makam Şoförü,Gıda,M,1
200904,Satış - Pazarlama,Aktif Satış Sorumlusu,Yapı,B,2


In [16]:
def get_support(data,feature_id,feature_val,cluster):
    n_cluster_size = len(cluster)
    num = 0
    for j in range(n_cluster_size):
        if data[cluster[j],feature_id] == feature_val:
            num = num+1
    return num

def similarity_instance_cluster(data,instance_id,cluster):
    n_instances,n_features = data.shape
    sim = 0.0

    for i in range(n_features):
        
        unique = []
        for j in range(len(cluster)):
            if data[cluster[j],i] not in unique:
                unique.append(data[cluster[j],i])
        temp = 0
        for j in range(len(unique)):
            temp = temp+get_support(data,i,unique[j],cluster)
        sim = sim+get_support(data,i,data[instance_id,i],cluster)*1.0/temp
    return sim

def squeezer(data,thre):
    # Initialize the clustering result
    label = [[0]]
    
    # Obtain the number of instances and features from input data
    n_instances,n_features = data.shape

    for i in range(1,n_instances):

        # Current number of clusters
        n_clusters = len(label)
        sim = [0]*n_clusters
        # Compute similarity between data[i,:] and each cluster
        for j in range(n_clusters):
            sim[j] = similarity_instance_cluster(data,i,label[j])
        
        sim_max = max(sim)

        for j in range(n_clusters):
            if sim[j] == sim_max:
                sim_max_cluster_id = j

        if sim_max>=thre:
            label[sim_max_cluster_id].append(i)
        else:
            label.append([i])

    return label

In [19]:
clusters = squeezer(data.to_numpy(),1)
len(clusters)

2